In [3]:
# CREATING INPUT - TARGET PAIRS 
import tiktoken

In [4]:

with open("the-verdict.txt","r",encoding="utf-8") as f:
    raw_text = f.read()
    
tokenizer =  tiktoken.get_encoding("gpt2")

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [5]:
enc_sample = enc_text[50:]

In [6]:
context_size = 4   #lenght of the input
#the context_size of 4 means that the model is trained to look at the sequence of 4 words (or tokens)
# to predict the next word in the sequence.
#the inout X is the first 4 tokens[1,2,3,4],and the target y is the next 4 tokens[2,3,4,5]

x=enc_sample[:context_size]
y=enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [7]:
for i in range(1 , context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context , "--->" , desired)

[290] ---> 4920
[290, 4920] ---> 2241
[290, 4920, 2241] ---> 287
[290, 4920, 2241, 287] ---> 257


In [8]:
for i in range(1 , context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context),"--->",tokenizer.decode([desired]))

 and --->  established
 and established --->  himself
 and established himself --->  in
 and established himself in --->  a


In [9]:
#IMPLEMENTING A DATA LOADER

#for the efficient data loader implementation , we will use pytorch's built-in dataset and dataloader classes.

In [10]:
!pip install torch
from torch.utils.data import Dataset , DataLoader

In [11]:
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []
        
        # tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        
        # use a sliding window to chunk the text
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i+max_length]
            target_chunk = token_ids[i+1:i+1+max_length]
            
            self.input_ids.append(input_chunk)
            self.target_ids.append(target_chunk)
    
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "target_ids": self.target_ids[idx]
        }


In [18]:
def create_dataloader_v1(txt,batch_size=4,max_length=256,stride=128,shuffle=True,drop_last=True,num_workers=0):


    #initialize tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")


    # create dataset
    dataset = GPTDatasetV1(txt , tokenizer , max_length , stride)

    #create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size = batch_size,
        shuffle = shuffle,
        drop_last = drop_last,
        num_workers = num_workers
    )

    return dataloader

In [19]:
with open("the-verdict.txt" , "r" , encoding="utf-8") as f:
    raw_text = f.read()

In [20]:
import torch

In [21]:
print("PyTorch version:" , torch.__version__)
dataloader =  create_dataloader_v1(
    raw_text , batch_size=1 , max_length=1 , stride=1 , shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

PyTorch version: 2.8.0+cpu
{'input_ids': [tensor([40])], 'target_ids': [tensor([367])]}
